# 特徴量として入れたいもの
- チャージャーフラグ
- ステージデータ
- ランクのチーム平均

In [1]:
from IPython.core.display import display, HTML 
display(HTML("<style>.container { width:100% !important; }</style>"))
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

%matplotlib inline

In [2]:
train = pd.read_csv("../data/input/train_data.csv")
test = pd.read_csv("../data/input/test_data.csv")

buki_data = pd.read_csv('../data/input/statink-weapon2.csv')
# buki_data.loc[buki_data['category2'] == 'maneuver', 'category2'] = 'maneuver_cat' # カテゴリ名称とブキ名称が被るためカテゴリ名称を変更する

In [3]:
train.head(3)

,id,period,game-ver,lobby-mode,lobby,mode,stage,A1-weapon,A1-rank,A1-level,...,B2-weapon,B2-rank,B2-level,B3-weapon,B3-rank,B3-level,B4-weapon,B4-rank,B4-level,y
0,1,2019-10-15T20:00:00+00:00,5.0.1,regular,standard,nawabari,sumeshi,sshooter_becchu,NaN,139,...,hokusai_becchu,NaN,26.0,herocharger_replica,NaN,68.0,sharp_neo,NaN,31.0,1
1,2,2019-12-14T04:00:00+00:00,5.0.1,regular,standard,nawabari,arowana,parashelter_sorella,NaN,198,...,squiclean_b,NaN,118.0,campingshelter,NaN,168.0,sputtery_clear,NaN,151.0,0
2,3,2019-12-25T14:00:00+00:00,5.0.1,gachi,standard,hoko,ama,nzap89,a-,114,...,nzap85,a+,163.0,prime_becchu,a-,160.0,dualsweeper_custom,a,126.0,0


In [4]:
test.head(3)

,id,period,game-ver,lobby-mode,lobby,mode,stage,A1-weapon,A1-rank,A1-level,...,B1-level,B2-weapon,B2-rank,B2-level,B3-weapon,B3-rank,B3-level,B4-weapon,B4-rank,B4-level
0,1,2019-12-17T12:00:00+00:00,5.0.1,gachi,standard,area,hakofugu,prime_collabo,x,174,...,160,nautilus47,x,151.0,l3reelgun_d,x,213.0,nzap89,x,306.0
1,2,2019-11-25T16:00:00+00:00,5.0.1,gachi,standard,asari,anchovy,prime_becchu,s+,363,...,326,dualsweeper_custom,s+,289.0,prime_becchu,s+,147.0,splatroller,s+,156.0
2,3,2019-10-22T08:00:00+00:00,5.0.1,gachi,standard,area,mutsugoro,furo_deco,s,116,...,76,momiji,s,232.0,nzap83,s,183.0,promodeler_pg,s,105.0


In [5]:
buki_data.head(3)

,category1,category2,key,subweapon,special,mainweapon,reskin,splatnet,[de-DE],[en-GB],...,[es-ES],[es-MX],[fr-CA],[fr-FR],[it-IT],[ja-JP],[nl-NL],[ru-RU],[zh-CN],[zh-TW]
0,shooter,shooter,52gal,pointsensor,sphere,52gal,52gal,50,.52 Gallon,.52 Gal,...,Salpicadora 2000,Salpicadora 2000,Calibre 2000,Calibre 2000,Calibro 2000,.52ガロン,.52 Kaliter,Струевик .52,点52口径重枪,點52口徑重槍
1,shooter,shooter,52gal_deco,curlingbomb,presser,52gal,52gal_deco,51,.52 Gallon Deko,.52 Gal Deco,...,Salpicadora 2000 DX,Salpicadora 2000 chic,Calibre 2000 chic,Calibre 2000 chic,Calibro 2000 DX,.52ガロンデコ,.52 Kaliter Deco,Струевик .52 «Деко»,点52口径重枪 Deco,點52口徑重槍 Deco
2,shooter,shooter,96gal,sprinkler,armor,96gal,96gal,80,.96 Gallon,.96 Gal,...,Salpicadora 3000,Salpicadora 3000,Calibre 3000,Calibre 3000,Calibro 3000,.96ガロン,.96 Kaliter,Струевик .96,点96口径重枪,點96口徑重槍


In [6]:
def team_level_agg(input_data):
    
    A_rank_col = ["A1-rank", "A2-rank", "A3-rank", "A4-rank"]
    B_rank_col = ["B1-rank", "B2-rank", "B3-rank", "B4-rank"]
    rank_col = A_rank_col + B_rank_col
    
    A_level_col = ["A1-level", "A2-level", "A3-level", "A4-level"]
    B_level_col = ["B1-level", "B2-level", "B3-level", "B4-level"]
    sel_col = rank_col + A_level_col + B_level_col


    # 高ランク順に数字を当てはめ
    for col in rank_col:
        rk_order = {'c-' : 1, 'c' : 2, 'c+' : 3, 'b-' : 4,'b' : 5, 'b+' : 6, 'a-' : 7, 'a' : 8, 'a+' : 9, 's' : 10, 's+' : 11, 'x' : 12}
        input_data[col] = input_data[col].map(rk_order)
    

    output_data =pd.DataFrame()
    
    output_data["A_rank_mean"] = input_data[A_rank_col].mean(axis = 1)
    output_data["A_rank_std"]  = input_data[A_rank_col].std(axis = 1)
    output_data["A_rank_max"]  = input_data[A_rank_col].max(axis = 1)
    output_data["A_rank_min"]  = input_data[A_rank_col].min(axis = 1)

    output_data["B_rank_mean"] = input_data[B_rank_col].mean(axis = 1)
    output_data["B_rank_std"]  = input_data[B_rank_col].std(axis = 1)
    output_data["B_rank_max"]  = input_data[B_rank_col].max(axis = 1)
    output_data["B_rank_min"]  = input_data[B_rank_col].min(axis = 1)    
    
    
    output_data["A_level_mean"] = input_data[A_level_col].mean(axis = 1)
    output_data["A_level_std"]  = input_data[A_level_col].std(axis = 1)
    output_data["A_level_max"]  = input_data[A_level_col].max(axis = 1)
    output_data["A_level_min"]  = input_data[A_level_col].min(axis = 1)

    output_data["B_level_mean"] = input_data[B_level_col].mean(axis = 1)
    output_data["B_level_std"]  = input_data[B_level_col].std(axis = 1)
    output_data["B_level_max"]  = input_data[B_level_col].max(axis = 1)
    output_data["B_level_min"]  = input_data[B_level_col].min(axis = 1)
    
    output_data = output_data.fillna(0)
    
    return output_data

In [7]:
train_feature_team_rank = team_level_agg(train)
test_feature_team_rank  = team_level_agg(test)

In [8]:
train_feature_team_rank.head(3)

,A_rank_mean,A_rank_std,A_rank_max,A_rank_min,B_rank_mean,B_rank_std,B_rank_max,B_rank_min,A_level_mean,A_level_std,A_level_max,A_level_min,B_level_mean,B_level_std,B_level_max,B_level_min
0,0.00,0.0,0.0,0.0,0.00,0.000000,0.0,0.0,70.0,68.102863,139.0,10.0,38.25,19.939492,68.0,26.0
1,0.00,0.0,0.0,0.0,0.00,0.000000,0.0,0.0,149.0,59.615434,198.0,77.0,130.00,37.585458,168.0,83.0
2,7.75,0.5,8.0,7.0,7.75,0.957427,9.0,7.0,128.5,67.441333,225.0,68.0,124.75,52.582475,163.0,50.0


In [9]:
test_feature_team_rank.head(3)

,A_rank_mean,A_rank_std,A_rank_max,A_rank_min,B_rank_mean,B_rank_std,B_rank_max,B_rank_min,A_level_mean,A_level_std,A_level_max,A_level_min,B_level_mean,B_level_std,B_level_max,B_level_min
0,12.0,0.0,12.0,12.0,12.0,0.0,12.0,12.0,166.00,49.564772,233.0,127.0,207.5,71.136020,306.0,151.0
1,11.0,0.0,11.0,11.0,11.0,0.0,11.0,11.0,170.75,153.350090,363.0,36.0,229.5,91.398395,326.0,147.0
2,10.0,0.0,10.0,10.0,10.0,0.0,10.0,10.0,136.50,29.938827,181.0,116.0,149.0,71.437618,232.0,76.0


In [10]:
def buki_cate_count_agg(input_data, buki_cate):
    A_weapon_col = ["A1-weapon", "A2-weapon", "A3-weapon", "A4-weapon"]
    B_weapon_col = ["B1-weapon", "B2-weapon", "B3-weapon", "B4-weapon"]
    buki_col = A_weapon_col + B_weapon_col
     
    buki_data = pd.read_csv('../data/input/statink-weapon2.csv')
    target_buki_list    = list(buki_data[buki_data["category2"] == buki_cate]["key"].unique())
    no_target_buki_list = list(buki_data[buki_data["category2"] != buki_cate]["key"].unique())
    
    
    output_data =pd.DataFrame()
    
    input_data = input_data[buki_col].replace(target_buki_list, 1).replace(no_target_buki_list, 0)
    
    output_data["A_charger_count"] =  input_data[A_weapon_col].sum(axis = 1)
    output_data["B_charger_count"] =  input_data[B_weapon_col].sum(axis = 1)
    
    return output_data

In [12]:
train_feat_charger_count = buki_cate_count_agg(train, "charger")
test_feat_charger_count  = buki_cate_count_agg(test,  "charger")

In [15]:
train_feat_charger_count.head(3)

,A_charger_count,B_charger_count
0,1.0,1.0
1,0.0,1.0
2,0.0,0.0


In [16]:
test_feat_charger_count.head(3)

,A_charger_count,B_charger_count
0,0.0,0.0
1,0.0,0.0
2,0.0,0.0


In [13]:
train_feat_charger_count.groupby("A_charger_count").count()

,B_charger_count
A_charger_count,
0.0,43915
1.0,20058
2.0,1991
3.0,154
4.0,7


In [14]:
train_feat_charger_count.groupby("B_charger_count").count()

,A_charger_count
B_charger_count,
0.0,41775
1.0,21971
2.0,2196
3.0,179
4.0,4


In [17]:
test_feat_charger_count.groupby("A_charger_count").count()

,B_charger_count
A_charger_count,
0.0,18690
1.0,8687
2.0,883
3.0,72
4.0,8


In [18]:
test_feat_charger_count.groupby("B_charger_count").count()

,A_charger_count
B_charger_count,
0.0,17804
1.0,9544
2.0,902
3.0,85
4.0,5
